# Hola Gerardo!

Mi nombre es David Bautista, soy code reviewer de TripleTen y voy a revisar el proyecto que acabas de desarrollar.

Cuando vea un error la primera vez, lo señalaré. Deberás encontrarlo y arreglarlo. La intención es que te prepares para un espacio real de trabajo. En un trabajo, el líder de tu equipo hará lo mismo. Si no puedes solucionar el error, te daré más información en la próxima ocasión.

Encontrarás mis comentarios más abajo - **por favor, no los muevas, no los modifiques ni los borres.**

¿Cómo lo voy a hacer? Voy a leer detenidamente cada una de las implementaciones que has llevado a cabo para cumplir con lo solicitado. Verás los comentarios de esta forma:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Si todo está perfecto.
</div>


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>


<div class="alert alert-block alert-danger">
    
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
    
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>


Puedes responderme de esta forma: 

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
</div>

¡Empecemos!

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

# Comentario General

Hola Gerardo, te felicito por el desarrollo del proyecto. Lograste completar las diferentes secciones de muy buena manera . </div> 

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Buen trabajo estructurando esta pequeña sección de introducción.  </div> 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from catboost import CatBoostClassifier
import lightgbm as lgb

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MaxAbsScaler

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo importando las librerías necesarias para el desarrollo del proyecto.  </div> 

## Preparación de datos

In [2]:
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo cargando y revisando de manera inicial el contenido del dataset.   </div> 

In [4]:
#df['DateCrawled'] = pd.to_datetime(df['DateCrawled'])
#df['DateCreated'] = pd.to_datetime(df['DateCreated'])
#df['LastSeen'] = pd.to_datetime(df['LastSeen'])

### Datos

POdemos observar que tiene 16 columnas, donde tenemos 6 datos categoricos y el resto son valores numericos.

Tenemos valores nulos en nuestras filas pero con la potenciacion deberia de no generar problema.

In [5]:
df[df.duplicated()].count()

DateCrawled          262
Price                262
VehicleType          256
RegistrationYear     262
Gearbox              259
Power                262
Model                258
Mileage              262
RegistrationMonth    262
FuelType             256
Brand                262
NotRepaired          253
DateCreated          262
NumberOfPictures     262
PostalCode           262
LastSeen             262
dtype: int64

Eliminamos los valores duplicados por que solo son relleno inecesario y hace que nuestros datos sean redundantes.

In [6]:
df= df.drop_duplicates()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354107 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354107 non-null  object
 1   Price              354107 non-null  int64 
 2   VehicleType        316623 non-null  object
 3   RegistrationYear   354107 non-null  int64 
 4   Gearbox            334277 non-null  object
 5   Power              354107 non-null  int64 
 6   Model              334406 non-null  object
 7   Mileage            354107 non-null  int64 
 8   RegistrationMonth  354107 non-null  int64 
 9   FuelType           321218 non-null  object
 10  Brand              354107 non-null  object
 11  NotRepaired        282962 non-null  object
 12  DateCreated        354107 non-null  object
 13  NumberOfPictures   354107 non-null  int64 
 14  PostalCode         354107 non-null  int64 
 15  LastSeen           354107 non-null  object
dtypes: int64(7), object(

In [8]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354107.000000,354107.000000,354107.000000,354107.000000,354107.000000,354107.0,354107.000000
mean,4416.433287,2004.235355,110.089651,128211.811684,5.714182,0.0,50507.145030
std,4514.338584,90.261168,189.914972,37906.590101,3.726682,0.0,25784.212094
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49406.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Aqui use el describe para ver valores atípicos y fuera de los limites, esto para reducir el numero de variables, y pues tambien ayuda a hacer mas eficiente nuestro codigo ya que no trabajamos con posibles datos falsos

In [9]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df_no_outliers = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


In [10]:
print(Q1)

Price                  1050.0
RegistrationYear       1999.0
Power                    69.0
Mileage              125000.0
RegistrationMonth         3.0
NumberOfPictures          0.0
PostalCode            30165.0
Name: 0.25, dtype: float64


In [11]:
df_no_outliers.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,282621.000000,282621.000000,282621.000000,282621.000000,282621.000000,282621.0,282621.000000
mean,3288.787383,2002.707605,102.191465,142656.384345,5.704955,0.0,50267.653147
std,3154.956258,6.113736,55.906518,16145.268530,3.741212,0.0,25653.110815
min,0.000000,1986.000000,0.000000,90000.000000,0.000000,0.0,1067.000000
25%,975.000000,1999.000000,68.000000,150000.000000,3.000000,0.0,29664.000000
50%,2200.000000,2002.000000,102.000000,150000.000000,6.000000,0.0,49090.000000
75%,4650.000000,2006.000000,140.000000,150000.000000,9.000000,0.0,70376.000000
max,14400.000000,2019.000000,254.000000,150000.000000,12.000000,0.0,99998.000000


<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Buen trabajo realizando esta exploración un poco más detallada del contenido del dataset.  </div> 

## Entrenamiento del modelo 

### Separacion de datos

In [12]:
df_Copy = df_no_outliers.copy()

In [13]:
features_Columns = df_Copy.select_dtypes(include=object).columns.tolist()

In [14]:
number_Columns = df_Copy.select_dtypes(include=int).columns.tolist()

In [15]:
df_Copy[features_Columns] = df_Copy[features_Columns].fillna('None')

In [16]:
df_Copy[number_Columns] = df_Copy[number_Columns].fillna(df_Copy[number_Columns].mean())

Realizamos un fill de los valores los cuales se encontraban vacios, los categoricos les pusimos None y los valores numericos nos fuimos por la media.

Esto por que hay filas que les falte un valor pueden sumar mucho al codigo

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Buen trabajo modificando temas como el de los valores nulos, aun así yo hubiera preferido que se realizara una exploración un poco más detallada de posibles soluciones a ese fenómeno en vez de simplemente eliminar los valores.  </div> 

### Vamos a codificar las variables categoricas


In [17]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(df_Copy), columns=df_Copy.columns)

In [18]:
data_ordinal.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
157021,9469.0,876.0,0.0,31.0,2.0,112.0,170.0,3.0,6.0,3.0,38.0,1.0,66.0,0.0,2219.0,3730.0
73991,3469.0,1333.0,2.0,16.0,2.0,146.0,207.0,3.0,5.0,7.0,0.0,1.0,26.0,0.0,5320.0,6115.0
105836,11446.0,788.0,5.0,17.0,2.0,71.0,101.0,3.0,11.0,7.0,31.0,1.0,78.0,0.0,1909.0,255.0
276196,349.0,635.0,6.0,15.0,2.0,71.0,83.0,3.0,12.0,7.0,24.0,1.0,2.0,0.0,3171.0,3083.0
60898,9556.0,940.0,0.0,30.0,2.0,114.0,11.0,3.0,11.0,0.0,2.0,1.0,66.0,0.0,1944.0,4228.0


Usamos un enconder ordinario, ya que si utilizabamos un OneHot teniamos muchas columnas lo que ocasionaba mucho consumo de memoria y que el Kernel fallara.

In [19]:
features_train, features_valid, target_train, target_valid = train_test_split(data_ordinal.drop('Price', axis=1), data_ordinal.Price, test_size=0.3, random_state=0)

Separación del codigo para entrenamiento y validación, utilizamos estas variables para todos para ahorrar consumo de memoria ya que este programa requiere de mucha memoria.

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo terminando de alistar las características y realizando el split de los datos.   </div> 

### Lightgmb

In [20]:
train_data =lgb.Dataset(features_train, label=target_train)

In [21]:
params = {
    'objective': 'regression', 
    'metric': 'l2',  
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}


In [22]:
model = lgb.train(params, train_data, num_boost_round=50)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.084212 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1427
[LightGBM] [Info] Number of data points in the train set: 197834, number of used features: 14
[LightGBM] [Info] Start training from score 997.319894


In [23]:
prediction = model.predict(features_valid)

In [24]:
MSE = mean_squared_error(target_valid,prediction)
RMSE = np.sqrt(MSE)

In [25]:
print('El error medio cuadratido es:',MSE,'\nLa raiz del error cuadratico es:',RMSE)

El error medio cuadratido es: 129245.42127556674 
La raiz del error cuadratico es: 359.50719224455963


No muestra un error cuadratico muy bajo y tambien es muy rapido.

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo desplegando y evaluando el desempeño del modelo Lightgmb. </div> 

### Bosque Aleatorio

In [26]:
model_forest = RandomForestClassifier(n_estimators=10,max_depth=1,random_state=0)

In [27]:
%%time
model_forest.fit(features_train,target_train)

CPU times: user 694 ms, sys: 0 ns, total: 694 ms
Wall time: 745 ms


RandomForestClassifier(max_depth=1, n_estimators=10, random_state=0)

In [28]:
%%time
predict_forest = model_forest.predict(features_valid)

CPU times: user 4.14 s, sys: 2.08 s, total: 6.23 s
Wall time: 6.24 s


In [29]:
MSE = mean_squared_error(target_valid,predict_forest)
RMSE = np.sqrt(MSE)

In [30]:
print('El error medio cuadratido es:',MSE,'\nLa raiz del error cuadratico es:',RMSE)

El error medio cuadratido es: 1506123.7102975692 
La raiz del error cuadratico es: 1227.2423193068146


En este caso el bosque aleatorio se le tuvieron que realizar muchas modificacion para que su consumo fuera menor ya que si se eleva un poco tumba el programa, aun asi mostro un error elevado y mucho tiempo de procesamiento.

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo desplegando, evaluando y estudiando las métricas de despliegue del Random Forest. </div> 

### Arbol de decisión

In [31]:
model_tree = DecisionTreeClassifier(max_depth=10)

In [32]:
model_tree.fit(features_train,target_train)

DecisionTreeClassifier(max_depth=10)

In [33]:
predict_tree = model_tree.predict(features_valid)

In [34]:
MSE = mean_squared_error(target_valid,predict_tree)
RMSE = np.sqrt(MSE)

In [35]:
print('El error medio cuadratido es:',MSE,'\nLa raiz del error cuadratico es:',RMSE)

El error medio cuadratido es: 256525.86416549707 
La raiz del error cuadratico es: 506.4838241893783


El arbol de decisión se comporta adecuadamente, a este si se le pudo aumentar el numero de profundida sin que el Kernel se callera, si tarda un poco en procesar pero es casi nulo y encontramos muy buenos resultados, el error cuadratico se sigue reduciendo.

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo desplegando y evaluando el desempeño del Decision Tree. </div> 

### Regresion lineal

In [36]:
model_lineal = LinearRegression().fit(features_train,target_train)

In [37]:
predict_lineal = model_lineal.predict(features_valid)

In [38]:
MSE = mean_squared_error(target_valid,predict_lineal)
RMSE = np.sqrt(MSE)


In [39]:
print('El error medio cuadratido es:',MSE,'\nLa raiz del error cuadratico es:',RMSE)

El error medio cuadratido es: 278448.92784709524 
La raiz del error cuadratico es: 527.6826014254168


El modelo de regresion lineal rapido y eficiente, con error bajo.

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Perfecto, buen trabajo desplegando y evaluando el desempeño de la Regresión. </div> 

## Análisis del modelo

El modelo con hiperparametros demostro su eficiencia, fue rapido y preciso, mas aparte de poder procesar una tabla con muchas columnas. Los demas modelos como el randomforest si mostro una deficiencia al hablarse de una tabla tan amplica, el decisiontree sigue siendo adecuado pero tiene un tiempo minimo. Regresion lineal mostro resultado rapidos y eficientes.

<div class="alert alert-block alert-success">
<b>Comentario del revisor </b> <a class="tocSkip"></a>

Buen trabajo. </div> 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos

### 